# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [74]:
# Import Python packages 

In [74]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [75]:
# checking your current working directory

In [75]:
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [76]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [77]:
# check the number of rows in your csv file

In [77]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [82]:
# To make a connection to a Cassandra instance 

In [83]:
from cassandra.cluster import Cluster
cluster = Cluster()

In [84]:
# To establish connection and begin executing queries, need a session

In [85]:
session = cluster.connect()

#### Create Keyspace

In [86]:
# Keyspace name is checked and if doesn't exist it is created

In [87]:
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [88]:
# Set keyspace

In [89]:
session.set_keyspace('sparkifydb')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [59]:
## Below code creates table for query that gives the artist, song title and song's length in the music app history that \
## was heard during sessionId = 338, and itemInSession = 4

In [90]:
session.execute("""CREATE TABLE IF NOT EXISTS song_length (sessionId varint, itemInSession varint, artist_name varchar,
song_title text, song_length float, PRIMARY KEY(sessionId, itemInSession));""")
             

In [60]:
# Code to load data into table from CSV file for Table song_length

In [91]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_length (sessionId, itemInSession, artist_name, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s) IF NOT EXISTS "
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [62]:
## CQL query that gives the artist, song title and song's length in the music app history that \
## was heard during sessionId = 338, and itemInSession = 4

In [93]:
result = session.execute("""SELECT artist_name, song_title, song_length FROM song_length 
            WHERE sessionId = 338 AND itemInSession = 4;""")
    
for row in result:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [63]:
##  Below code creates table for query that gives the following: name of artist, song (sorted by itemInSession)\
## and user (first and last name) for userid = 10, sessionid = 182

In [94]:
session.execute("""CREATE TABLE IF NOT EXISTS song_playlist_session (userid varint, sessionid varint, 
iteminsession varint, artist_name varchar, song_title text, listener_lastname text, listener_firstname text, 
PRIMARY KEY ((userId, sessionId), itemInSession))""")               

In [64]:
## Load data into session_table from CSV for Table song_playlist_session

In [95]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (userid, sessionid, iteminSession, song_title, artist_name,\
        listener_firstname, listener_lastname)"  
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s) IF NOT EXISTS "
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9],  line[0], line[1], line[4]))
                    

In [66]:
## CQL query that gives the following: name of artist, song (sorted by itemInSession) and user (first and last name)\ 
##for userid = 10, sessionid = 182

In [96]:
result_2 = session.execute("""SELECT artist_name, song_title, listener_firstname, 
listener_lastname FROM song_playlist_session WHERE userid = 10 AND sessionid = 182;""")
    
for row in result_2:
    print (row.artist_name, row.song_title, row.listener_firstname, row.listener_lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [68]:
## Below code creates table for query that gives every user name (first and last) in my music app history\
## who listened to the song 'All Hands Against His Own'

In [97]:
session.execute("""CREATE TABLE IF NOT EXISTS song_listener_history (song_title text, userid varint,
listener_firstname text, listener_lastname text,  PRIMARY KEY (song_title, userid));""")

In [69]:
## Load data into session_table from CSV for Table song_listener_history

In [98]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listener_history (song_title, userId, listener_firstname, listener_lastname)"  
        query = query + "VALUES (%s, %s, %s, %s) IF NOT EXISTS "
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [71]:
## CQL query that gives every user name (first and last) in my music app history who listened\
## to the song 'All Hands Against His Own'

In [99]:
result_3 = session.execute("""SELECT listener_firstname, listener_lastname FROM song_listener_history 
WHERE song_title = 'All Hands Against His Own';""")
for row in result_3:
    print (row.listener_firstname, row.listener_lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## Code to Drop the table before closing out the sessions

In [100]:
table_list = ['song_length','song_playlist_session','song_listener_history']

for table in table_list:
    session.execute("""DROP TABLE {0};""".format(table))

### Close the session and cluster connection¶

In [101]:
session.shutdown()
cluster.shutdown()